In [8]:
import torch
from torch.utils.data import DataLoader, TensorDataset

def make_dataloader(n):
    X = np.random.random((n, 2)) * 2 - 1
    Y = (X[:, 0] > X[:, 1] + 0.25).astype(int) + 1

    X = torch.tensor(X, dtype=torch.float)
    Y = torch.tensor(Y, dtype=torch.long)

    div1 = int(n*0.8)
    div2 = int(n*0.9)
    Xs = [X[:div1], X[div1:div2], X[div2:]]
    Ys = [Y[:div1], Y[div1:div2], Y[div2:]]

    dataset = TensorDataset(Xs[0], Ys[0])
    data_loader = DataLoader(dataset, batch_size=4)
    return data_loader

In [9]:
import torch.nn as nn
from metal.mmtl.task import Task

foo_input = nn.Linear(2, 10)
bar_input = foo_input #nn.Linear(100, 7)

foo_head = nn.Linear(10, 2)
bar_head = nn.Linear(10, 2)

foo = Task("foo_task", [make_dataloader(5000), None, None], foo_input, foo_head)
bar = Task("bar_task", [make_dataloader(2000), None, None], bar_input, bar_head)
# baz = Task("baz_task", "baz_head", [make_dataloader(100), None, None])
tasks = [foo, bar]

In [10]:
from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

model = MetalModel(tasks)
# trainer = MultitaskTrainer()
# trainer.train_model(model, tasks, n_epochs=1)


Network architecture:
MetalModel(
  (input_modules): ModuleDict(
    (bar_task): Linear(in_features=2, out_features=10, bias=True)
    (foo_task): Linear(in_features=2, out_features=10, bias=True)
  )
  (head_modules): ModuleDict(
    (bar_task): Linear(in_features=10, out_features=2, bias=True)
    (foo_task): Linear(in_features=10, out_features=2, bias=True)
  )
  (task_paths): ModuleDict(
    (bar_task): Sequential(
      (0): Linear(in_features=2, out_features=10, bias=True)
      (1): Linear(in_features=10, out_features=2, bias=True)
    )
    (foo_task): Sequential(
      (0): Linear(in_features=2, out_features=10, bias=True)
      (1): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)



In [13]:
for batch in foo.data_loaders[0]:
    X, Y = batch
    print(model(X, ['foo_task']))
    print(model.predict_proba(X, ['foo_task']))
    print(model.calculate_loss(X, Y, ['foo_task']))    
    break

tensor([2, 2, 2, 2])
{'foo_task': tensor([[-0.0398,  0.2911],
        [ 0.1936,  0.3616],
        [ 0.0255,  0.3249],
        [ 0.2031,  0.3740]], grad_fn=<AddmmBackward>)}
{'foo_task': tensor([[0.4180, 0.5820],
        [0.4581, 0.5419],
        [0.4257, 0.5743],
        [0.4574, 0.5426]], grad_fn=<SoftmaxBackward>)}


/Users/bradenjh/repos/metal/metal/mmtl/metal_model.py:83: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return {t: self.probs_hat_funcs[t](out) for t, out in self.forward(X, task_names).items()}


RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at /Users/soumith/mc3build/conda-bld/pytorch_1549310147607/work/aten/src/THNN/generic/ClassNLLCriterion.c:93

In [5]:
len([p for p in model.parameters()])
# for p in model.parameters():
#     print(p)

6

In [6]:
import torch.nn as nn

amod = nn.Linear(2,7)
# bmod = nn.Linear(2,7)
bmod = amod

In [7]:
amod == bmod

True